In [ ]:
import numpy as np
import scipy
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
import pandas as pd
#import plotly.express as px

In [ ]:
sns.set()

## Factory data

The following dataset contains quality control measurements from 6
machines in a factory (range: 0-120, units of the measurements are irrelevant here). 

In the dataset, each column contains the measurements for a single machine. Quality control measurements are expensive and time-consuming, so only 5 measurements were done for each machine. 

In addition to the existing machines, we are interested in the quality of another machine (the seventh machine) which is not in the dataset.

In [ ]:
df_wide = pd.read_csv("factory.csv")
df_wide.set_index("measurement", inplace=True)
df_wide.columns.name = "machine"
df_wide

* Transform the dataset to the long format: one row per machine per measurement

A "long" format is more convenient for the following analyises. Let us *melt* the dataframe!

In [ ]:
df = pd.melt(df_wide.reset_index(),
             id_vars=["measurement"],
             value_vars=['M1', 'M2', 'M3', 'M4', 'M5', 'M6'],
             value_name='quality')
df["machine"] = df["machine"].astype("category") # useful for group analysis (with pymc3)

In [ ]:
df.head(6)

Note: the category column has the useful properies `cat.categories` (all the possible values of the categorical variable) and `cat.codes` (an equivalent integer representation of the variable)

In [ ]:
df["machine"].cat.categories

In [ ]:
df["machine"].cat.codes.head(6)

## Some traditional data analysis (data-challenge style)

* Obtain the global mean and standard deviation of the quality measurement (pooled mean/standard deviation)

In [ ]:
df["quality"].mean(), df["quality"].std()

* Plot a histogram and a boxplot of the quality measurement

In [ ]:
sns.histplot(df["quality"]);
#px.histogram(df["quality"]);

In [ ]:
sns.boxplot(df["quality"]);

* Obtain the mean and standard deviation of quality measurement, for the different machines (unpooled mean/standard deviation)

In [ ]:
df.groupby("machine")[["quality"]].agg(["mean", "std"])

* Draw boxplots of quality for the different factories

In [ ]:
sns.boxplot(data=df, x="quality", y="machine");
#px.box(df, x="quality", y="machine")

Even from a classical analysis, there seems to be evidence that different machines have different quality.

## Pooled Bayesian Model

Consider the following *pooled* Bayesian model:
\begin{align*}
\mu &\sim N(90, 10) \\
\sigma &\sim \text{HalfNormal}(36) \\
\vec{y}_{ij} &\sim N(\mu, \sigma) \\
\end{align*}


* Discuss the pooled model and its underlying hypotheses 

The model does not make distinction between different machines. It is a single normal-normal model for measurements from all the different machines; it assumes the measurement to share *global* mean and standard deviation $\mu$ and $\sigma$, respectively. 

It could have been constructed assuming that: 
1. The (global) mean quality $\mu$ is between $90-3*10=60$ and $90+3*10=120$ with probability 99%
2. All quality measurements $\vec{y}_{ij}$ is in a range of width $25*6=150$ with probability 99%


In [ ]:
pd.DataFrame(stats.halfnorm.rvs(size=10_000, scale=36)).median()

* Implement the pooled model in pymc3

In [ ]:
with pm.Model() as model_pooled:
    global_mu = pm.Normal('global_mu', mu=90, sigma=15)
    global_sigma = pm.HalfNormal('global_sigma', sigma=30)
    y = pm.Normal('y', mu=global_mu, sigma=global_sigma, observed=df["quality"])
    trace_pooled = pm.sample(10_000, idata_kwargs = {'log_likelihood': True})

In [ ]:
pm.model_to_graphviz(model_pooled)

In [ ]:
with model_pooled:
    display(az.summary(trace_pooled))

In [ ]:
with model_pooled:
    az.plot_posterior(trace_pooled);

## Unpooled Bayesian Model

Consider the following Bayesian unpooled model
\begin{align*}
\mu_{j} &\sim {N}(90, 20) \\
\sigma_{j} &\sim \text{HalfNormal}(23) \\
\vec{y}_{ij} &\sim {N}(\mu_j, \sigma_j) \\
\end{align*}

* Discuss the unpooled model and its underlying hypotheses 

The unpooled model actually consists in 6 completely separate normal-normal sub-models for the 6 different machines. 
It could have build according to the hipotheses that: 
1. The mean quality of each machine is between $90-3*20=30$ and $90+3*20=150$ with probability 99%
2. The quality measurements of each machine are in a range of width $15*6=90$ with probability 99%

In [ ]:
pd.DataFrame(stats.halfnorm.rvs(size=10_000, scale=23)).median()

* Implement the unpooled model in pymc3

In [ ]:
df["machine_id"] = df["machine"].cat.codes
machines = list(df["machine"].cat.categories)
n_machines = len(machines) # number of machines (6)
machines, n_machines

In [ ]:
with pm.Model() as model_unpooled:
    group_mu = pm.Normal('group_mu', mu=90, sigma=20, shape=n_machines)
    group_sigma = pm.HalfNormal('group_sigma', sigma=23, shape=n_machines)
    #y = pm.Normal('y', mu=group_mu[df["machine_id"]], sd=group_sigma[df["machine_id"]], observed=df["quality"])
    y = pm.Normal('y', mu=group_mu[df["machine"].cat.codes],
                       sigma=group_sigma[df["machine"].cat.codes],
                       observed=df["quality"])
    trace_unpooled = pm.sample(10_000, idata_kwargs = {'log_likelihood': True})


In [ ]:
pm.model_to_graphviz(model_unpooled)

In [ ]:
trace_unpooled.posterior = trace_unpooled.posterior.assign_coords(group_mu_dim_0=machines,
                                                                  group_sigma_dim_0=machines)

In [ ]:
with model_unpooled:
    display(az.summary(trace_unpooled))

In [ ]:
with model_unpooled:
    axes = az.plot_forest (trace_unpooled,  kind='ridgeplot', hdi_prob=0.95, var_names="group_mu", combined=True);
    plt.title("Posterior mean for the 6 machines")
    #axes[0].set_yticklabels(machines[::-1])
    

In [ ]:
with model_unpooled:
    axes = az.plot_forest (trace_unpooled,  kind='ridgeplot', hdi_prob=0.95, var_names="group_sigma", combined=True);
    plt.title("Posterior standard deviation for the 6 machines")
    #axes[0].set_yticklabels(machines[::-1])

* Compare the sample unpooled standard deviation with the bayesian estimate. What do you observe?

We already computed the sample standard deviations before:

In [ ]:
df.groupby("machine")[["quality"]].agg(["mean", "std"])

The posterior means of the bayesian estimates are:

In [ ]:
with model_unpooled:
    display(az.summary(trace_unpooled, var_names=["group_sigma"]))

In [ ]:
# az.plot_posterior(trace_separate["group_std"][:, 0], point_estimate='mode')
with model_unpooled:
    az.plot_posterior(trace_unpooled, var_names=["group_sigma"], point_estimate='mode'); # the posterior mean is 29, the mode is 19!

The sample std is rather different from the posterior std mean. The sample std is actually much similar to the posterior mode (as expected from theory)! The posterior std distributions are heavily right-skewed!

## Hierarchical model, common std

Consider the following Bayesian hierarchical model:

\begin{align*}
\mu_\mu & \sim N(90, 10)\\
\sigma_\mu  & \sim \text{HalfNormal}(30) \\
\mu_{j} &\sim {N}(\mu_\mu, \sigma_\mu) \\
\sigma &\sim \text{HalfNormal}(23) \\
\vec{y}_{ij} &\sim {N}(\mu_j, \sigma) \\
\end{align*}



* Discuss the hierarchical model and its underlying hypotheses 

The hierarchical model may be seen as an extension of the unpooled model. While in the unpooled model the 6 different $\mu_j$ are independent of each other, in the hierarchical model they share common parent variabes $\mu_\mu$ and $\sigma_\mu$ representing our prior belief about their mean value and their spread, respectively. Indeed, each $\mu_j$ is assumed to be normal with mean $\mu_\mu$ and standard deviation $\sigma_\mu$. Furthermore, we assume a single random variable $\sigma$ modeling the spread of the different measurements for a given machine (note that in the previous unpooled model we had different standard deviation parameters for the different groups, it is kind of an arbitrary choice/modeling assumption).

From the numerical values of the model coefficients, the modeler might have made the following assumptions:
1. The mean of the mean quality of each machine is between $90-3*10=60$ and $90+3*10=120$ with probability 99% $\Rightarrow$ from $\mu_\mu \sim N(90, 10)$
2. The quality measurements of each machine are in a range of width $15*6=90$ with probability 99% $\Rightarrow$ from $\sigma \sim \text{HalfNormal}(23)$
3. The different $\mu_j$ vary in a range of width $6*20=120$ with probability 99% $\Rightarrow$ from $\sigma_\mu  \sim \text{HalfNormal}(30)$

In [ ]:
pd.DataFrame(stats.halfnorm.rvs(size=10_000, scale=23)).median()

In [ ]:
pd.DataFrame(stats.halfnorm.rvs(size=10_000, scale=30)).median()

* Implement the hierarchical model in pymc3

In [ ]:
with pm.Model() as model_hierarchical:
    
    # hyper_priors
    mu_mu = pm.Normal('mu_mu', mu=90, sigma=10)
    sigma_mu = pm.HalfNormal('sigma_mu', sigma=30)

    # priors    
    group_mu = pm.Normal('group_mu', mu=mu_mu, sigma=sigma_mu, shape=n_machines)
    global_std = pm.HalfNormal('global_std', sigma=23)
    y = pm.Normal('y', mu=group_mu[df["machine_id"]],
                       sigma=global_std, observed=df["quality"])
    trace_hierarchical = pm.sample(10_000, idata_kwargs = {'log_likelihood': True})

In [ ]:
pm.model_to_graphviz(model_hierarchical)

In [ ]:
trace_hierarchical.posterior = trace_hierarchical.posterior.assign_coords(group_mu_dim_0=machines)

In [ ]:
az.summary(trace_hierarchical)

## Model selection 

What is the best model ultimately? In our case: Pooled, Unpooled, or Hierarchical?


To answer this question, we need a **model selection** strategy/metric.

* You know metrics for point regression models (mse, rmse, mae,...)
* You know metrics for point classification models (accuracy, prediciton, recall...)


### The WAIC Criterion 
To score full Bayesian models, we need a metric that evaluates a (sample-based approximation of a) *distribution*. 

* A popular metric to score a distribution is the log-pointwise-predictive-density ${\rm lpd}(y, \Theta)$:
$$ {\rm lpd}(y, \Theta) = \sum_i \log \frac{1}{S} \sum_s p(y_i | \Theta_s),$$
where $y$ are the observations and $\Theta$ is the sample-based approximation of the posterior ($\Theta_s$ is a single MCMC draw). It is the (approximate) logarithm of the expected likelihood (the higher, the better)!


* The ${\rm lpd}(y, \Theta)$ generally increases for increasing model complexity and it is thus prone to overfitting when used on the same data used for model building. It should be evaluated with fresh data in (cross)-validation.

* The WAIC criterion (the lower, the better) balances high ${\rm lpd}$ and low model complexity explicitly. It can be applied directly on the training data $y$!

\begin{align}
{\rm WAIC}(y, \Theta) &= -2(\rm{lpd} - p_{\rm waic}).\\
 p_{{\rm waic}} &= \sum_i {\rm var}_{\theta} \log p(y_i|\theta)
\end{align}

* The WAIC is equivalent up to a factor to the ${\rm elpd}$, the *expected* $\rm{lpd}$ on fresh data (the higher, the better):

$${\rm elpd}(y, \Theta) = \rm{lpd} - p_{\rm waic}.$$

Advantages of WAIC/elpd:

* Well-defined both for continuous and for categorical observations
* Applicable to Bayesian models returning point-wise estimates
* Applicable to the training data directly (built-in complexity penalty)
* Works well in practice

Note: WAIC stands for Widely Applicable Information Criterion!

### The WAIC Criterion in arviz

Arviz has a built-in function `az.waic` to compute the WAIC (actually, as of v0.12.1, arviz computes the ${\rm elpd}$, see <a href="https://python.arviz.org/en/v0.12.1/api/generated/arviz.waic.html?highlight=waic">documentation</a>):


In [ ]:
az.waic(trace_pooled)

There is an even more convenient method to evaluate the criterion on several models and rank them according to the WAIC criterion (from best to worst, ascending WAIC/descending elpd)

In [ ]:
#with factory_pooled, factory_separate, factory_hierarchical:
comp_df = az.compare({"model_pooled": trace_pooled,
                      "model_unpooled": trace_unpooled,
                      "model_hierarchical": trace_hierarchical},
                      ic="waic") # ic stands for "information criterion"
comp_df

In [ ]:
az.plot_compare(comp_df);

## Posterior predictions of new measurements

* Given the pooled model, make a prediction for a new measurement

We can do it by adding a line in pymc3 corresponding to the prediction:

In [ ]:
with pm.Model() as model_pooled_with_pred:
    
    global_mu = pm.Normal('global_mu', mu=90, sigma=15)
    global_sigma = pm.HalfNormal('global_sigma', sigma=30)
    y = pm.Normal('y', mu=global_mu, sigma=global_sigma, observed=df["quality"])
    
    y_pred = pm.Normal("y_pred", mu=global_mu, sigma=global_sigma)
    
    trace_pooled_with_pred = pm.sample(10_000, return_inferencedata=True)

In [ ]:
with model_pooled_with_pred:
    display(az.summary(trace_pooled_with_pred)) # var_names='y_pred'

Otherwise, we can play with the trace directly. For each sampled value of global_mu and global_sigma, we sample from
N(global_mu, global_sigma)

In [ ]:
post_pooled = az.extract(trace_pooled.posterior)

y_pred = np.zeros_like(post_pooled.global_mu.values)
for idx in range(y_pred.shape[0]):
    y_pred[idx] = scipy.stats.norm.rvs(loc=post_pooled.global_mu.values[idx],
                                       scale=post_pooled.global_sigma.values[idx])
    
#y_pred = scipy.stats.norm.rvs(loc=post_pooled.global_mu.values,scale=post_pooled.global_sigma.values) # faster vectorized implementation!

y_pred.mean(), y_pred.std() # similar to y_pred in the trace (note: there is randomness!)

* Given the unpooled model, make a prediction for a new measurement of the machine M1

In [ ]:
with pm.Model() as model_unpooled_with_pred_M1:
    group_mu = pm.Normal('group_mu', mu=90, sigma=20, shape=n_machines)
    group_sigma = pm.HalfNormal('group_sigma', sigma=23, shape=n_machines)
    y = pm.Normal('y', mu=group_mu[df["machine"].cat.codes],
                  sigma=group_sigma[df["machine"].cat.codes],
                  observed=df["quality"])
    
    y_M1_pred = pm.Normal('y_M1', mu=group_mu[0], sigma=group_sigma[0])
    
    trace_unpooled_with_pred_M1 = pm.sample(10_000, return_inferencedata=True)


In [ ]:
with model_unpooled_with_pred_M1:
    display(az.summary(trace_unpooled_with_pred_M1, var_names="y_M1"))

In [ ]:
post_unpooled = az.extract(trace_unpooled.posterior)
y_M1_pred = scipy.stats.norm.rvs(loc=post_unpooled.group_mu[0].values,
                                 scale=post_unpooled.group_sigma[0].values)
y_M1_pred.mean(), y_M1_pred.std() # similar to y_pred in the trace (note: there is randomness!)